In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [6]:
# scintific computing libraries
import pandas as pd                                      
import numpy as np                    
from scipy import optimize, stats        

# visualisation libraries
import matplotlib.pyplot as plt                      
import seaborn as sns                

# algorithmic library
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import svm                 
from sklearn.linear_model import LogisticRegression                              
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, RandomizedSearchCV, GridSearchCV  
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, log_loss, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.pipeline import Pipeline

%matplotlib inline

In [4]:
path = 'data/'
df_train = pd.read_csv(path+'sales_train.csv')
df_shops = pd.read_csv(path+'shops.csv')
df_items = pd.read_csv(path+'items.csv')
df_item_categories = pd.read_csv(path+'item_categories.csv')
df_test = pd.read_csv(path+'test.csv')
df_sample_submission = pd.read_csv(path'sample_submission.csv')

In [8]:
df_sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [9]:
print(df_train.shape)
df_train.head()

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
print(df_shops.shape)
df_shops.head()

(60, 2)


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [6]:
print(df_items.shape)
df_items.head()

(22170, 3)


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
print(df_item_categories.shape)
df_item_categories.head()

(84, 2)


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [8]:
print(df_test.shape)
df_test.head()

(214200, 3)


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [10]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [9]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [10]:
df_train.dtypes

date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

In [11]:
df_train.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [12]:
df_train.drop(['date_block_num','item_price'], axis=1, inplace=True)

In [13]:
df_train['date'] = pd.to_datetime(df_train['date'])
df = df_train.groupby([df_train['date'].apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).sum()
df = df.pivot_table(index=['item_id','shop_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df.head()

date,item_id,shop_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_test = pd.merge(df_test, df, on=['item_id','shop_id'], how='left')
df_test.drop(['ID', '2013-01'], axis=1, inplace=True)
df_test = df_test.fillna(0)
df_test.head()

,shop_id,item_id,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,...,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12
0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0
1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0
3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
'''df_train1 = df_train.groupby(['year','month','shop_id','item_id']).sum()
df_train1.reset_index(inplace=True)
df_train1.drop(['month','year','item_price'], axis=1, inplace=True)
df_train1.rename(columns = {'item_cnt_day':'item_cnt_month'}, inplace=True)
df_train1.head()'''

"df_train1 = df_train.groupby(['year','month','shop_id','item_id']).sum()\ndf_train1.reset_index(inplace=True)\ndf_train1.drop(['month','year','item_price'], axis=1, inplace=True)\ndf_train1.rename(columns = {'item_cnt_day':'item_cnt_month'}, inplace=True)\ndf_train1.head()"

## Model Development

In [16]:
Y_train = df['2015-12'].values
X_train = df.drop(['2015-12'], axis = 1)
X_test = df_test

print(X_train.shape, Y_train.shape)
print(X_test.shape)

(424124, 37) (424124,)
(214200, 37)


In [17]:
'''X_train = StandardScaler().fit(X_train).transform(X_train)
X_test = StandardScaler().fit(X_test).transform(X_test)'''

'X_train = StandardScaler().fit(X_train).transform(X_train)\nX_test = StandardScaler().fit(X_test).transform(X_test)'

In [18]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=4)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (339299, 37) (339299,)
Test set: (84825, 37) (84825,)


In [19]:
LR = LinearRegression()
LR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, LR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, LR.predict(x_test)))
print('Test set score:', LR.score(x_train,y_train))

Train set mse: 0.09102679143170209
Test set mse: 0.4326311869790391
Test set score: 0.6304376953136375


In [20]:
RFR = RandomForestRegressor(n_estimators = 50)
RFR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, RFR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, RFR.predict(x_test)))
print('Test set score:', RFR.score(x_train,y_train))

Train set mse: 0.019354223855655342
Test set mse: 0.4771081072796936
Test set score: 0.9214232264917478


In [21]:
'''from lightgbm import LGBMRegressor
model=LGBMRegressor(n_estimators=200, learning_rate=0.03, max_depth=8)
model.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, model.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, model.predict(x_test)))
print('Test set score:', model.score(x_train,y_train))'''

"from lightgbm import LGBMRegressor\nmodel=LGBMRegressor(n_estimators=200, learning_rate=0.03, max_depth=8)\nmodel.fit(x_train,y_train)\n\nprint('Train set mse:', mean_squared_error(y_train, model.predict(x_train)))\nprint('Test set mse:', mean_squared_error(y_test, model.predict(x_test)))\nprint('Test set score:', model.score(x_train,y_train))"

In [22]:
'''GBR = GradientBoostingRegressor(n_estimators = 50 , max_depth = 3)
GBR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, GBR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, GBR.predict(x_test)))
print('Test set score:', GBR.score(x_train,y_train))'''

"GBR = GradientBoostingRegressor(n_estimators = 50 , max_depth = 3)\nGBR.fit(x_train,y_train)\n\nprint('Train set mse:', mean_squared_error(y_train, GBR.predict(x_train)))\nprint('Test set mse:', mean_squared_error(y_test, GBR.predict(x_test)))\nprint('Test set score:', GBR.score(x_train,y_train))"

In [23]:
prediction = RFR.predict(X_test)

In [24]:
prediction = list(map(round, prediction))

## Submission

In [25]:
print(df_sample_submission.shape)
df_sample_submission.head()

(214200, 2)


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [26]:
df_sample_submission['item_cnt_month'] = prediction
df_sample_submission.to_csv('prediction.csv', index=False)
df_sample_submission.head()

,ID,item_cnt_month
0,0,1.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0
